In [350]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import csv
import math
import pickle
import random
from collections import defaultdict

# **Read Inputs**

In [351]:
# Read user data
lastWord = ""
words = defaultdict(dict)
with open('Data/Example/example_data.txt', 'r') as input:
    for line in input:
        touchpoints = [0,0]
        line.strip
        if line.startswith("==="):
            words[str(line)[3:len(str(line))-4]] = []
            lastWord = str(line)[3:len(str(line))-4]
        else:
            tempPoints = str(line)[0:len(str(line))-2].split(" ")
            touchpoints[0] = float(tempPoints[1])
            touchpoints[1] = float(tempPoints[2])
            words[lastWord].append(touchpoints)


defaultdict(dict,
            {'Word 0': [['a', 4.819348, 13.72294]],
             'Word 1': [['p', 34.008873, 8.6187],
              ['r', 12.373077, 8.34533],
              ['o', 28.881908, 8.07189],
              ['b', 19.790089, 16.5029],
              ['l', 31.889727, 13.17607],
              ['e', 8.92092, 7.616163],
              ['m', 28.266672, 16.68519]],
             'Word 2': [['w', 6.0498195, 9.21122],
              ['i', 26.96784, 7.935174],
              ['t', 16.47465, 9.53023],
              ['h', 22.114313, 13.22164]],
             'Word 3': [['t', 16.269571, 8.07189],
              ['h', 22.55865, 13.40393],
              ['e', 9.809595, 9.48465]],
             'Word 4': [['e', 10.253932, 8.6187],
              ['n', 24.985413, 16.22946],
              ['g', 17.97856, 13.26722],
              ['i', 27.207098, 7.069287],
              ['n', 25.600649, 17.82452],
              ['e', 9.809595, 8.57319]],
             'Word 5': [['p', 31.753008, 8.52762],
              [

In [352]:
# Read dictionary
with open('Data/unigram.dict', 'rb') as unigramModelFile:
    unigramModel = pickle.load(unigramModelFile)
unigramModelFile.close()

# Read keyboard data
keyboard_raw = pd.read_csv("Data/keyboard.csv")
keyboard = keyboard_raw[['key', 'x_mm', 'y_mm']]
keyboard

   key       x_mm       y_mm
0    a   4.025010   9.625024
1    b  18.900047  13.650034
2    c  12.950032  13.650034
3    d   9.975025   9.625024
4    e   8.487521   5.600014
5    f  12.950032   9.625024
6    g  15.925039   9.625024
7    h  18.900047   9.625024
8    i  23.362558   5.600014
9    j  21.875053   9.625024
10   k  24.850061   9.625024
11   l  27.825070   9.625024
12   m  24.850061  13.650034
13   n  21.875053  13.650034
14   o  26.337564   5.600014
15   p  29.312572   5.600014
16   q   2.537506   5.600014
17   r  11.462528   5.600014
18   s   7.000017   9.625024
19   t  14.437536   5.600014
20   u  20.387550   5.600014
21   v  15.925039  13.650034
22   w   5.512514   5.600014
23   x   9.975025  13.650034
24   y  17.412542   5.600014
25   z   7.000017  13.650034
26      12.950032  17.675043


# **Unigram Language Model Decoder**

In [353]:
# Keyboard size and dual Gaussian model parameters
key_width = 3
key_height = 4
a = 2.403
b = 0.017
c = 2.295
d = 0.016

def get_likelihood(p, mu, sigma):
    """
    TODO: Calculate the likelihood that a touch point p is from the 2D Gaussian distribution N(mu, sigma)
    """
    #lik = 1/((math.pow(2*math.pi*math.pow(sigma,2)), 1/2)) * np.exp(-(1/(2*math.pow(sigma,2)))(math.pow(p-mu,2)))
  
    lik = a * math.exp(-(math.pow((p[0]-mu[0]),2)/(2*(a+b*math.pow(key_width,2)))+(math.pow(p[1]-mu[1],2)/(2*(c+d*math.pow(key_height,2))))))
    return lik          

def is_letter(p, letter):
    """
    TODO: Determine if touch point p is located inside the boundary of the key: letter
    """

    tmp = keyboard.loc[keyboard['key'] == letter]
    x_mm = tmp.iloc[0]['x_mm']
    y_mm = tmp.iloc[0]['y_mm']
    if p[0] < (x_mm + key_width) and p[0] > x_mm and p[1] > (y_mm - key_height) and p[1] < y_mm:
        print(p, "is",letter)
        return True
    return False 

def get_literal_string(touchpoints):
    """ 
    TODO: Compute the literal string using is_letter(p, letter) method for a collection of touch points that represents a word. 
          If a touch point does not fall inside any key boundary, use '?' to represent the corresponding character.
    """
    literal_string = ""
    tmp = keyboard['key']
    for touchpoint in touchpoints:
        found = False
        for i in range(25):
            letter = tmp.values[i]
            if is_letter(touchpoint, letter):
                literal_string += letter
                found = True
        if found == False:
            literal_string+= "?"
    print("literal String:",literal_string)
    return literal_string

In [354]:
possible_words = []
length = len(touchpoints)
firstTP = touchpoints[0]
lastTP = touchpoints[len(touchpoints)-1]
possible_words = dict(filter(lambda x: len(str(x[0])) == length, unigramModel.items()))
#print(possible_words)

tmp = keyboard.loc[keyboard['key'] == 'c']
x_mm = tmp.iloc[0]['x_mm']
y_mm = tmp.iloc[0]['y_mm']
print(tmp.iloc[0]['x_mm'])


12.950032


In [355]:
def unigram_lm_decoder(touchpoints):
    """
    A language decoder that uses the dual Gaussian touch point spatial disrtibution model and a unigram language model.
    Input: a list/collection of touch points that represents a certain word
    Output: the decoded word for the input
    
    TODO: Step a --- Get all possible words and their corresponding probabilities from the dictionary. 
          Use the length of the correct word to filter possible words
          You may also use the first and/or the last touchpoint to further narrow down possible words
    """

    """ sum =0
    for k in range(p):
        x = -(1)/(2* math.pow(sigma,2))*math.pow(p[k]-mu,2)
        sum += (math.pow(x, k))/(math.factorial(k))

    lik = 1/((math.pow(2*math.pi*math.pow(sigma,2)), 1/2)) * sum
    """
    possible_words = []
    length = len(touchpoints)
    firstTP = touchpoints[0]
    lastTP = touchpoints[len(touchpoints)-1]
    possible_words = dict(filter(lambda x: len(str(x[0])) == length, unigramModel.items()))
    #possible_words = dict(filter(lambda x: is_letter(firstTP, x[0][:1]), unigramModel.items()))
    print("list sorted! total items:", len(possible_words))
    
    # Calculate p(w|s_1, s_2, ... s_n) ~ p(s_1, s_2, ..., s_n|w)*p(w) = \Pi(p(s_i|c_i))p(w) for each possible word
    p_w_s = []                  # Holds p(w|s_1, s_2, ..., s_n) for all possible words
    for item in possible_words:
        word = item                  # The current possible word
        p_w =  possible_words[item]  # Probability of the current possible word in the unigram language model
        p_s_w = 1                 # Holds p(s_1, s_2, ..., s_n|w) for the current possible word

        for j, letter in enumerate(list(word)):
            currentPoints = touchpoints[j]
            # TODO: Step b --- Apply the spatial model to get p(s_i|c_i)
            tmp = keyboard.loc[keyboard['key'] == letter]
            x_mm = tmp.iloc[0]['x_mm']
            y_mm = tmp.iloc[0]['y_mm']
            mu = [x_mm + (0.5*key_width), y_mm - (0.5*key_height)] # mu = center of key ci

            sigma =[a+b*math.pow(key_width,2),c+d*math.pow(key_height,2)] # [sigma_X, sigma_Y]

            p_s_c = get_likelihood([currentPoints[0], currentPoints[1]], mu, sigma)

            # TODO: Step b --- Multiply the current p(s_i|c_i) to p(s_1, s_2, ..., s_n|W)
            p_s_w *= p_s_c
            #print("current Psw for letter",letter,":", p_s_w)
        
        # TODO: Step c --- Calculate p(w|s_1, s_2, ... s_n) from p(s_1, s_2, ..., s_n|w) and p(w). Append the result to list
        p_w_s.append([word,p_w * p_s_w])
        #print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    # TODO: Step d ---- Choose word by the maximum of p(w|s_1, s_2, ..., s_n)
    decoded_word = [0,0]
    for i in p_w_s:
        if(i[1] > decoded_word[1]):
            decoded_word = i
    print("decoded word:",decoded_word)
    return decoded_word

In [356]:
decoded_success_count = 0
literal_success_count = 0
decoded_words = []
literal_strings = []
correct_words = []
for word in words:
    touchpoints = words[word]
    """
    TODO: Use above methods to compute the correct word, decoded word, and the literal string for each touch point collection
          Append results to the corresponding list
          Update the decoded words/literal strings success count. 
              --- If the decoded word/literal string is the same as correct word, increase 1 to decoded words/literal strings success count
    """
    print("trying word:",word)
    decoded_words.append(unigram_lm_decoder(touchpoints))
    literal_strings.append(get_literal_string(touchpoints))
# TODO: calculate the success rate for both the decoded words and the literal strings using the docoded word/literal string success count

# TODO: Write to results.txt
with open("results.txt", 'w') as output:
    # The first line: success_rate(decoded_words), success_rate(literal_strings)
    for i in range(len(decoded_words)):
        output.write(decoded_words[i], "\n")
    for i in range(len(correct_words)):
        # Each line after: correct_word, decoded_word, literal_string
        break;
    output.close()